In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
from torch_geometric.data import HeteroData
from sklearn.preprocessing import StandardScaler

# Carichiamo il dataset
file_path = '/content/filtered_data_con_etichetta.csv'
dataset = pd.read_csv(file_path)

# 1. Creiamo le mappature uniche di pazienti e malattie
paziente_diagnosi_malattia = dataset[['CODICE_FISCALE_ASSISTITO', 'ICD9_CM']]
unique_pazienti = paziente_diagnosi_malattia['CODICE_FISCALE_ASSISTITO'].unique()
unique_malattie = paziente_diagnosi_malattia['ICD9_CM'].unique()

# Mappature per indici
paziente_id_mapping = {code: idx for idx, code in enumerate(unique_pazienti)}
malattia_id_mapping = {code: idx for idx, code in enumerate(unique_malattie)}

# Convertiamo in indici
pazienti_indices = paziente_diagnosi_malattia['CODICE_FISCALE_ASSISTITO'].map(paziente_id_mapping).to_numpy()
malattie_indices = paziente_diagnosi_malattia['ICD9_CM'].map(malattia_id_mapping).to_numpy()

# 2. Creiamo il grafo eterogeneo
data = HeteroData()

# Aggiungiamo i nodi
data['paziente'].num_nodes = len(unique_pazienti)
data['malattia'].num_nodes = len(unique_malattie)

# Aggiungiamo gli edge index per la relazione paziente-malattia (diagnosi)
data['paziente', 'diagnosi', 'malattia'].edge_index = torch.tensor([pazienti_indices, malattie_indices], dtype=torch.long)
data['malattia', 'diagnosi', 'paziente'].edge_index = torch.tensor([malattie_indices, pazienti_indices], dtype=torch.long)

scaler_pazienti = StandardScaler()
scaler_malattie = StandardScaler()


# 3. Preprocessare le feature dei pazienti
def process_pazienti_features(df):
    df['SESSO'] = df['SESSO'].map({'M': 0, 'F': 1})  # Codifica del sesso
    df['SESSO'] = df['SESSO'].fillna(df['SESSO'].mode()[0])  # Riempimento dei NaN in 'SESSO' con la moda
    df['DATA_PRESCRIZIONE'] = pd.to_datetime(df['DATA_PRESCRIZIONE'])
    df['ETÀ'] = df['DATA_PRESCRIZIONE'].dt.year - df['ANNO_NASCITA']  # Calcolo dell'età
    df['ETÀ'] = df['ETÀ'].fillna(df['ETÀ'].median())  # Riempimento dei NaN
    pazienti_features = scaler_pazienti.fit_transform(df[['SESSO', 'ETÀ']])  # Normalizzazione
    return pazienti_features

# Preprocessare le feature delle malattie
def process_malattie_features(df):
    df['ICD9_CM_CODE'] = df['ICD9_CM'].astype('category').cat.codes  # Codifica ICD9
    df['ETICHETTA_CODE'] = df['etichetta'].astype('category').cat.codes  # Codifica etichetta
    df['ICD9_CM_CODE'] = df['ICD9_CM_CODE'].fillna(df['ICD9_CM_CODE'].median())  # Riempimento dei NaN
    df['ETICHETTA_CODE'] = df['ETICHETTA_CODE'].fillna(df['ETICHETTA_CODE'].median())  # Riempimento dei NaN
    malattie_features = scaler_malattie.fit_transform(df[['ICD9_CM_CODE', 'ETICHETTA_CODE']])  # Normalizzazione
    return malattie_features

# 4. Preprocessiamo e aggiungiamo le feature dei pazienti e delle malattie al grafo
# Raggruppiamo i pazienti per CODICE_FISCALE_ASSISTITO
pazienti_unique = dataset.groupby('CODICE_FISCALE_ASSISTITO').agg({
    'SESSO': 'first',
    'ANNO_NASCITA': 'first',
    'DATA_PRESCRIZIONE': 'max'
}).reset_index()
pazienti_unique['idx'] = pazienti_unique['CODICE_FISCALE_ASSISTITO'].map(paziente_id_mapping)
pazienti_unique = pazienti_unique.sort_values('idx')

# Feature dei pazienti
pazienti_features = process_pazienti_features(pazienti_unique)
data['paziente'].x = torch.tensor(pazienti_features, dtype=torch.float)

# Preprocessiamo le feature delle malattie
malattie_unique = dataset[['ICD9_CM', 'etichetta']].drop_duplicates()
malattie_unique['idx'] = malattie_unique['ICD9_CM'].map(malattia_id_mapping)
malattie_unique = malattie_unique.sort_values('idx')

# Feature delle malattie
malattie_features = process_malattie_features(malattie_unique)
data['malattia'].x = torch.tensor(malattie_features, dtype=torch.float)

# Rimozione nodi paziente con NaN
valid_pazienti_indices = ~torch.isnan(data['paziente'].x).any(dim=1)
valid_pazienti_ids = torch.arange(data['paziente'].num_nodes)[valid_pazienti_indices]
data['paziente'].x = data['paziente'].x[valid_pazienti_indices]

# Aggiorna gli edge_index per i pazienti
mask_paziente_edges = torch.isin(data['paziente', 'diagnosi', 'malattia'].edge_index[0], valid_pazienti_ids)
data['paziente', 'diagnosi', 'malattia'].edge_index = data['paziente', 'diagnosi', 'malattia'].edge_index[:, mask_paziente_edges]

# Rimozione nodi malattia con NaN
valid_malattie_indices = ~torch.isnan(data['malattia'].x).any(dim=1)
valid_malattie_ids = torch.arange(data['malattia'].num_nodes)[valid_malattie_indices]
data['malattia'].x = data['malattia'].x[valid_malattie_indices]

# Aggiorna gli edge_index per le malattie
mask_malattia_edges = torch.isin(data['paziente', 'diagnosi', 'malattia'].edge_index[1], valid_malattie_ids)
data['paziente', 'diagnosi', 'malattia'].edge_index = data['paziente', 'diagnosi', 'malattia'].edge_index[:, mask_malattia_edges]

# Aggiorna il numero di nodi
data['paziente'].num_nodes = data['paziente'].x.size(0)
data['malattia'].num_nodes = data['malattia'].x.size(0)

# Verifiche finali
print(f"Numero di nodi pazienti: {data['paziente'].num_nodes}, Feature shape: {data['paziente'].x.shape}")
print(f"Numero di nodi malattie: {data['malattia'].num_nodes}, Feature shape: {data['malattia'].x.shape}")
print(f"Max indice paziente: {pazienti_indices.max()}, Numero di nodi paziente: {data['paziente'].num_nodes}")
print(f"Max indice malattia: {malattie_indices.max()}, Numero di nodi malattia: {data['malattia'].num_nodes}")


<ipython-input-2-f143e4c4e5c7>:31: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  data['paziente', 'diagnosi', 'malattia'].edge_index = torch.tensor([pazienti_indices, malattie_indices], dtype=torch.long)


Numero di nodi pazienti: 1244, Feature shape: torch.Size([1244, 2])
Numero di nodi malattie: 2114, Feature shape: torch.Size([2114, 2])
Max indice paziente: 1243, Numero di nodi paziente: 1244
Max indice malattia: 2113, Numero di nodi malattia: 2114


In [3]:
print(data)

HeteroData(
  paziente={
    num_nodes=1244,
    x=[1244, 2],
  },
  malattia={
    num_nodes=2114,
    x=[2114, 2],
  },
  (paziente, diagnosi, malattia)={ edge_index=[2, 999927] },
  (malattia, diagnosi, paziente)={ edge_index=[2, 999927] }
)


In [4]:
import random
import numpy as np
from torch_geometric.utils import negative_sampling

# Funzione per dividere gli archi in training, test e validation set
def train_test_split_edges(data, test_ratio=0.15, val_ratio=0.15):
    # Prendiamo gli edge_index per la relazione paziente-malattia
    edge_index = data['paziente', 'diagnosi', 'malattia'].edge_index

    # Numero totale di archi
    num_edges = edge_index.size(1)

    # Definiamo quanti archi mettere in test e validation
    num_test = int(num_edges * test_ratio)
    num_val = int(num_edges * val_ratio)

    # Shuffling casuale degli indici degli archi
    perm = torch.randperm(num_edges)
    edge_index = edge_index[:, perm]

    # Dividiamo gli archi
    test_edges = edge_index[:, :num_test]
    val_edges = edge_index[:, num_test:num_test + num_val]
    train_edges = edge_index[:, num_test + num_val:]

    return train_edges, val_edges, test_edges


In [5]:
# Funzione per generare archi negativi (coppie di nodi non collegate)
def generate_negative_edges(data, num_neg_edges, edge_index):
    neg_edge_index = negative_sampling(
        edge_index=edge_index,
        num_nodes=(data['paziente'].num_nodes, data['malattia'].num_nodes),
        num_neg_samples=num_neg_edges,
        force_undirected=False  # Questo dipende dalla direzione delle tue relazioni
    )
    return neg_edge_index


In [6]:
# Eseguiamo il train-test split
train_edges, val_edges, test_edges = train_test_split_edges(data)

# Numero di archi negativi da generare (pari agli archi positivi in ogni set)
num_train_neg = train_edges.size(1)
num_val_neg = val_edges.size(1)
num_test_neg = test_edges.size(1)

# Generiamo archi negativi per training, validation e test set
train_neg_edges = generate_negative_edges(data, num_train_neg, train_edges)
val_neg_edges = generate_negative_edges(data, num_val_neg, val_edges)
test_neg_edges = generate_negative_edges(data, num_test_neg, test_edges)

# Costruiamo gli edge_label per il training
train_edge_index = torch.cat([train_edges, train_neg_edges], dim=1)
train_edge_label = torch.cat([torch.ones(train_edges.size(1)), torch.zeros(train_neg_edges.size(1))])

# Costruiamo gli edge_label per il validation
val_edge_index = torch.cat([val_edges, val_neg_edges], dim=1)
val_edge_label = torch.cat([torch.ones(val_edges.size(1)), torch.zeros(val_neg_edges.size(1))])

# Costruiamo gli edge_label per il test
test_edge_index = torch.cat([test_edges, test_neg_edges], dim=1)
test_edge_label = torch.cat([torch.ones(test_edges.size(1)), torch.zeros(test_neg_edges.size(1))])

print(f"Train edges: {train_edge_index.shape}, Train labels: {train_edge_label.shape}")
print(f"Validation edges: {val_edge_index.shape}, Validation labels: {val_edge_label.shape}")
print(f"Test edges: {test_edge_index.shape}, Test labels: {test_edge_label.shape}")


Train edges: torch.Size([2, 1399898]), Train labels: torch.Size([1399898])
Validation edges: torch.Size([2, 299978]), Validation labels: torch.Size([299978])
Test edges: torch.Size([2, 299978]), Test labels: torch.Size([299978])


In [7]:
import torch
from torch_geometric.nn import HeteroConv, SAGEConv, Linear
import torch.nn.functional as F

# Definiamo il modello con Dropout
class HeteroGNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, dropout=0.2):
        super(HeteroGNN, self).__init__()

        self.dropout = dropout  # Probabilità di dropout

        # Primo livello di convoluzione eterogenea
        self.conv1 = HeteroConv({
            ('paziente', 'diagnosi', 'malattia'): SAGEConv((-1, -1), hidden_channels),
            ('malattia', 'diagnosi', 'paziente'): SAGEConv((-1, -1), hidden_channels)
        }, aggr='mean')

        # Secondo livello di convoluzione eterogenea
        self.conv2 = HeteroConv({
            ('paziente', 'diagnosi', 'malattia'): SAGEConv((-1, -1), hidden_channels),
            ('malattia', 'diagnosi', 'paziente'): SAGEConv((-1, -1), hidden_channels)
        }, aggr='mean')

        # Terzo livello di convoluzione
        self.conv3 = HeteroConv({
            ('paziente', 'diagnosi', 'malattia'): SAGEConv((-1, -1), hidden_channels),
            ('malattia', 'diagnosi', 'paziente'): SAGEConv((-1, -1), hidden_channels)
        }, aggr='mean')

        # Layer finale per la predizione dei link
        self.decoder = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        # Primo livello di convoluzione con Dropout
        x_dict = self.conv1(x_dict, edge_index_dict)
        x_dict = {key: F.dropout(F.relu(x), p=self.dropout, training=self.training) for key, x in x_dict.items()}

        # Secondo livello di convoluzione con Dropout
        x_dict = self.conv2(x_dict, edge_index_dict)
        x_dict = {key: F.dropout(F.relu(x), p=self.dropout, training=self.training) for key, x in x_dict.items()}

        # Terzo livello di convoluzione con Dropout
        x_dict = self.conv3(x_dict, edge_index_dict)
        x_dict = {key: F.dropout(F.relu(x), p=self.dropout, training=self.training) for key, x in x_dict.items()}

        return x_dict

    def decode(self, z_dict, edge_label_index):
        src, dst = edge_label_index
        paziente_emb = z_dict['paziente'][src]
        malattia_emb = z_dict['malattia'][dst]
        return torch.sigmoid((paziente_emb * malattia_emb).sum(dim=-1))

In [21]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Ipotizziamo che hidden_channels = 64 e out_channels = 1 (per probabilità di link)
model = HeteroGNN(hidden_channels=64, out_channels=1)

# Funzione di perdita (Binary Cross Entropy per link prediction)
loss_fn = torch.nn.BCELoss()

# Definizione dell'ottimizzatore
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Definizione dello scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=True)


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [22]:
import torch
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score

def train(model, data, train_edge_index, train_edge_label, optimizer, loss_fn):
    model.train()  # Modalità training
    optimizer.zero_grad()  # Azzeriamo i gradienti

    # Forward pass
    z_dict = model(data.x_dict, data.edge_index_dict)
    pred = model.decode(z_dict, train_edge_index)

    # Calcoliamo la perdita
    loss = loss_fn(pred, train_edge_label.float())

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Calcoliamo l'accuratezza
    pred_label = (pred > 0.5).float()
    acc = accuracy_score(train_edge_label.cpu(), pred_label.cpu())

    return loss.item(), acc

def evaluate(model, data, edge_index, edge_label):
    model.eval()  # Modalità evaluation
    with torch.no_grad():
        # Forward pass
        z_dict = model(data.x_dict, data.edge_index_dict)
        pred = model.decode(z_dict, edge_index)

    # Calcoliamo l'accuratezza
    pred_label = (pred > 0.5).float()
    acc = accuracy_score(edge_label.cpu(), pred_label.cpu())

    return acc

def test(model, data, edge_index, edge_label):
    model.eval()  # Modalità evaluation
    with torch.no_grad():
        # Forward pass
        z_dict = model(data.x_dict, data.edge_index_dict)
        pred = model.decode(z_dict, edge_index)

    # Calcoliamo varie metriche
    pred_label = (pred > 0.5).float()
    auc = roc_auc_score(edge_label.cpu(), pred.cpu())
    acc = accuracy_score(edge_label.cpu(), pred_label.cpu())
    precision = precision_score(edge_label.cpu(), pred_label.cpu())
    recall = recall_score(edge_label.cpu(), pred_label.cpu())
    f1 = f1_score(edge_label.cpu(), pred_label.cpu())

    # Stampiamo tutte le metriche
    print(f"Test AUC: {auc:.4f}")
    print(f"Test Accuracy: {acc:.4f}")
    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")
    print(f"Test F1 Score: {f1:.4f}")

    return auc, acc, precision, recall, f1

# Parametri di addestramento
num_epochs = 300
loss_fn = torch.nn.BCELoss()

# Ciclo di addestramento
for epoch in range(1, num_epochs + 1):
    # Addestramento su train set
    train_loss, train_acc = train(model, data, train_edge_index, train_edge_label, optimizer, loss_fn)

    # Aggiorniamo lo scheduler
    scheduler.step(train_loss)

    # Ogni 10 epoche eseguiamo una valutazione su train e validation set
    if epoch % 10 == 0:
        val_acc = evaluate(model, data, val_edge_index, val_edge_label)
        print(f"Epoch {epoch}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}, Validation Accuracy: {val_acc:.4f}")

# Dopo l'addestramento, valutiamo il set di test e stampiamo le metriche finali
print("\nFinal Test Evaluation:")
test_auc, test_acc, test_precision, test_recall, test_f1 = test(model, data, test_edge_index, test_edge_label)


Epoch 10/300, Train Loss: 0.6945, Train Accuracy: 0.5083, Validation Accuracy: 0.5744
Epoch 20/300, Train Loss: 0.6873, Train Accuracy: 0.5370, Validation Accuracy: 0.5572
Epoch 30/300, Train Loss: 0.6709, Train Accuracy: 0.5557, Validation Accuracy: 0.6293
Epoch 40/300, Train Loss: 0.6640, Train Accuracy: 0.6216, Validation Accuracy: 0.6431
Epoch 50/300, Train Loss: 0.6352, Train Accuracy: 0.6423, Validation Accuracy: 0.6430
Epoch 60/300, Train Loss: 0.6338, Train Accuracy: 0.6770, Validation Accuracy: 0.7107
Epoch 70/300, Train Loss: 0.6017, Train Accuracy: 0.7088, Validation Accuracy: 0.7519
Epoch 80/300, Train Loss: 0.6090, Train Accuracy: 0.7128, Validation Accuracy: 0.7654
Epoch 90/300, Train Loss: 0.5953, Train Accuracy: 0.7191, Validation Accuracy: 0.7612
Epoch 100/300, Train Loss: 0.5748, Train Accuracy: 0.7483, Validation Accuracy: 0.7686
Epoch 110/300, Train Loss: 0.5885, Train Accuracy: 0.7459, Validation Accuracy: 0.7796
Epoch 120/300, Train Loss: 0.5876, Train Accuracy: 0